# Segmenting and Clustering Neighborhoods in Toronto - Part 3

#### Scroll down for Part 3. 

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

- Start by creating a new Notebook for this assignment.

- Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

- To create the above dataframe:

    - The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
    - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [1]:
# Import the required libraries
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium # map rendering library
import requests


import numpy as np
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors


### Part 1 - Scrape the webpage and build the data frame

In [2]:
# Url 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"


In [3]:
# Get the data and process with beautiful soup.
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

# In the page find the right table: this is the wiki sortable one.
right_table=soup.find('table', class_='wikitable sortable')

In [4]:
# Loop through the table and for each row get the postal code, borough and neighborhood and append it 
# to a dataframe. 

column_names = ['Postalcode','Borough','Neighborhood']
neighborhoods = pd.DataFrame(columns=column_names)
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        post_code = cells[0].find(text=True).strip('\n')
        borough = cells[1].find(text=True).strip('\n')
        nbr = cells[2].find(text=True).strip('\n')
        neighborhoods = neighborhoods.append({'Postalcode': post_code,
                                              'Borough': borough,
                                              'Neighborhood': nbr
                                              }, ignore_index=True)

In [5]:
neighborhoods.head()

Postalcode           Borough               Neighborhood
0        M1A      Not assigned               Not assigned
1        M2A      Not assigned               Not assigned
2        M3A        North York                  Parkwoods
3        M4A        North York           Victoria Village
4        M5A  Downtown Toronto  Regent Park, Harbourfront

In [6]:
neighborhoods=neighborhoods[neighborhoods['Borough']!='Not assigned']
neighborhoods.head()

Postalcode           Borough                                 Neighborhood
2        M3A        North York                                    Parkwoods
3        M4A        North York                             Victoria Village
4        M5A  Downtown Toronto                    Regent Park, Harbourfront
5        M6A        North York             Lawrence Manor, Lawrence Heights
6        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government

In [7]:
neighborhoods.shape

(103, 3)

In [8]:
len(neighborhoods['Postalcode'].unique())

103

The wiki page has been modified so that the postal codes are unique in the tables. So we dont have to bother with the section on merging neighborhoods by postal code. Therefore I have omitted this step.

### Number of rows of Data Frame

In [9]:
neighborhoods.shape[0]

103

##  Part 2

Use the csv file to create a merged data frame with the latitude andlongitude.

In [10]:
# Read in the csv file
loc_df = pd.read_csv('http://cocl.us/Geospatial_data')
loc_df.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [11]:
loc_df.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Merge the dataframes on Postal Code. To do that rename the column from "Postal Code" to "Postalcode"

In [12]:
loc_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
merged_df = pd.merge(loc_df, neighborhoods, on='Postalcode')
merged_df=merged_df[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [13]:
merged_df.head()

Postalcode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

### Part 3

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

- to add enough Markdown cells to explain what you decided to do and to report any observations you make.
- to generate maps to visualize your neighborhoods and how they cluster together.

In [14]:
# First get a list of unique Boroughs
merged_df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [15]:
# Get the corodinates of Toonto
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with Neighborhoods imposed on top.

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

I am going to be focusing on the "Toronto" neighborhood. So let us create a new dataframe for Yourk data.

In [17]:
toronto_data = merged_df[merged_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

Postalcode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

Now Let us get the geographical coordinates of Toronto

In [18]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Next would be defining the credentials for Foursquare. This is removed during publishing.

### Explore neighborhoods of Toronto

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [22]:
print(toronto_venues.shape)
toronto_venues.head()

(1621, 7)


Neighborhood  Neighborhood Latitude  \
0                   The Beaches              43.676357   
1                   The Beaches              43.676357   
2                   The Beaches              43.676357   
3                   The Beaches              43.676357   
4  The Danforth West, Riverdale              43.679557   

   Neighborhood Longitude                               Venue  Venue Latitude  \
0              -79.293031                   Glen Manor Ravine       43.676821   
1              -79.293031  The Big Carrot Natural Food Market       43.678879   
2              -79.293031                 Grover Pub and Grub       43.679181   
3              -79.293031                       Upper Beaches       43.680563   
4              -79.352188                       MenEssentials       43.677820   

   Venue Longitude     Venue Category  
0       -79.293942              Trail  
1       -79.297734  Health Food Store  
2       -79.297215                Pub  
3       -79.292869       Neighborhood  
4       -79.351265     Cosmetics Shop

In [23]:
#Let's find out how many unique categories can be curated from all the returned venues

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 unique categories.


Let's check how many venues were returned for each neighborhood

In [24]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            57   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     68   
Christie                                                               16   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           14   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          24   
India Bazaar, The Beaches West                                         22   
Kensington Market, Chinatown, Grange Park                              60   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               44   
Moore Park, Summerhill East                                             1   
North Toronto West,  Lawrence Park                                     20   
Parkdale, Roncesvalles                                                 15   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              45   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     37   
St. James Town                                                         79   
St. James Town, Cabbagetown                                            46   
Stn A PO Boxes                                                         97   
Studio District                                                        41   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    20   
The Beaches                                                             4   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         37   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             57   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing Centre, South Ce...                      16   
CN Tower, King and Spadina, Railway Lands, Harb...                      16   
Central Bay Street                                                      68   
Christie                                                                16   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              35   
Davisville North                    

### Analyze each neighborhood

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  \
0            0                  0        0                   0   
1            0                  0        0                   0   
2            0                  0        0                   0   
3            0                  0        0                   0   
4            0                  0        0                   0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  ...  Toy / Game Store  Trail  Train Station  \
0             0         0  ...                 0      1              0   
1             0         0  ...                 0      0              0   
2             0         0  ...                 0      0              0   
3             0         0  ...                 0      0              0   
4             0         0  ...                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Video Store  \
0                              0                 0            0   
1                              0                 0            0   
2                              0                 0            0   
3                              0                 0            0   
4                              0                 0            0   

   Vietnamese Restaurant  Wine Bar  Wine Shop  Women's Store  
0                      0         0          0              0  
1                      0         0          0              0  
2                      0         0          0              0  
3                      0         0          0              0  
4                      0         0          0              0  

[5 rows x 236 columns]

In [26]:
toronto_onehot.shape

(1621, 236)

In [27]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                                        Berczy Park     0.000000   
1       Brockton, Parkdale Village, Exhibition Place     0.000000   
2  Business reply mail Processing Centre, South C...     0.062500   
3  CN Tower, King and Spadina, Railway Lands, Har...     0.000000   
4                                 Central Bay Street     0.014706   

   Afghan Restaurant  Airport  Airport Food Court  Airport Lounge  \
0                0.0   0.0000              0.0000           0.000   
1                0.0   0.0000              0.0000           0.000   
2                0.0   0.0000              0.0000           0.000   
3                0.0   0.0625              0.0625           0.125   
4                0.0   0.0000              0.0000           0.000   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  ...  \
0            0.000             0.000                  0.0           0.0  ...   
1            0.000             0.000                  0.0           0.0  ...   
2            0.000             0.000                  0.0           0.0  ...   
3            0.125             0.125                  0.0           0.0  ...   
4            0.000             0.000                  0.0           0.0  ...   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0               0.0    0.0            0.0                       0.017544   
1               0.0    0.0            0.0                       0.000000   
2               0.0    0.0            0.0                       0.000000   
3               0.0    0.0            0.0                       0.000000   
4               0.0    0.0            0.0                       0.014706   

   Video Game Store  Video Store  Vietnamese Restaurant  Wine Bar  Wine Shop  \
0               0.0     0.000000                    0.0  0.000000        0.0   
1               0.0     0.000000                    0.0  0.000000        0.0   
2               0.0     0.000000                    0.0  0.000000        0.0   
3               0.0     0.000000                    0.0  0.000000        0.0   
4               0.0     0.014706                    0.0  0.014706        0.0   

   Women's Store  
0            0.0  
1            0.0  
2            0.0  
3            0.0  
4            0.0  

[5 rows x 236 columns]

Let's sort the venues in descending order

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...  Gym / Fitness Center   
3  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar            Restaurant                Bakery   
1        Breakfast Spot           Coffee Shop                Bakery   
2            Smoke Shop         Auto Workshop               Brewery   
3       Airport Service      Airport Terminal           Coffee Shop   
4    Italian Restaurant   Japanese Restaurant        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0              Beer Bar    Seafood Restaurant           Cheese Shop   
1          Intersection                   Bar    Italian Restaurant   
2         Burrito Place            Comic Shop        Farmers Market   
3              Boutique   Rental Car Location                 Plane   
4                  Café           Salad Place                   Bar   

  8th Most Common Venue 9th Most Common Venue  10th Most Common Venue  
0                  Café             Irish Pub                Pharmacy  
1            Restaurant          Climbing Gym  Furniture / Home Store  
2  Fast Food Restaurant                Garden           Garden Center  
3         Boat or Ferry                   Bar         Harbor / Marina  
4       Bubble Tea Shop          Burger Joint        Department Store

### Cluster Neighborhoods

In [30]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [31]:
# Add labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

Postalcode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0                 Trail     Health Food Store   
1 -79.352188               0      Greek Restaurant    Italian Restaurant   
2 -79.315572               0  Fast Food Restaurant                  Park   
3 -79.340923               0                  Café           Coffee Shop   
4 -79.388790               2                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0                   Pub      Doner Restaurant          Dessert Shop   
1           Coffee Shop             Bookstore        Ice Cream Shop   
2           Pizza Place                   Pub             Pet Store   
3   American Restaurant                Bakery               Brewery   
4              Bus Line      Department Store           Event Space   

    6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
0                   Diner        Discount Store          Distribution Center   
1  Furniture / Home Store            Restaurant         Caribbean Restaurant   
2       Fish & Chips Shop        Ice Cream Shop             Sushi Restaurant   
3               Gastropub  Gym / Fitness Center                  Fish Market   
4    Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

  9th Most Common Venue 10th Most Common Venue  
0               Dog Run          Women's Store  
1                   Pub                   Café  
2               Brewery     Italian Restaurant  
3             Pet Store                   Park  
4            Donut Shop       Doner Restaurant

In [32]:
neighborhoods_venues_sorted.head()

Cluster Labels                                       Neighborhood  \
0               0                                        Berczy Park   
1               0       Brockton, Parkdale Village, Exhibition Place   
2               0  Business reply mail Processing Centre, South C...   
3               0  CN Tower, King and Spadina, Railway Lands, Har...   
4               0                                 Central Bay Street   

  1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
0           Coffee Shop          Cocktail Bar            Restaurant   
1                  Café        Breakfast Spot           Coffee Shop   
2  Gym / Fitness Center            Smoke Shop         Auto Workshop   
3        Airport Lounge       Airport Service      Airport Terminal   
4           Coffee Shop    Italian Restaurant   Japanese Restaurant   

  4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
0                Bakery              Beer Bar    Seafood Restaurant   
1                Bakery          Intersection                   Bar   
2               Brewery         Burrito Place            Comic Shop   
3           Coffee Shop              Boutique   Rental Car Location   
4        Sandwich Place                  Café           Salad Place   

  7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
0           Cheese Shop                  Café             Irish Pub   
1    Italian Restaurant            Restaurant          Climbing Gym   
2        Farmers Market  Fast Food Restaurant                Garden   
3                 Plane         Boat or Ferry                   Bar   
4                   Bar       Bubble Tea Shop          Burger Joint   

   10th Most Common Venue  
0                Pharmacy  
1  Furniture / Home Store  
2           Garden Center  
3         Harbor / Marina  
4        Department Store

In [33]:

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters